In [2]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# PyTorch Workflow
#    1: "data (prepare and load)",
#    2: "build model",
#    3: "fitting the model to data (training)",
#    4: "making predictions and evaluating a model (inference)",
#    5: "saving and loading a model",
#    6: "putting it all together"


In [4]:
# 1. Data Train & Load

data = pd.read_csv('IRIS.csv') # import dataframe as pandas

datanp = data.to_numpy() # convert pandas to numpy, shuffle data
np.random.shuffle(datanp) 

# Split into testing, training data & convert to PyTorch Tensors
trainX, trainY, testX, testY = datanp[0:140, 0:3].astype('float32'), datanp[0:140, 3].astype('float32'), datanp[140:, 0:3].astype('float32'), datanp[140:, 3].astype('float32')
trainX, trainY, testX, testY = torch.from_numpy(trainX), torch.from_numpy(trainY), torch.from_numpy(testX), torch.from_numpy(testY)
testY = testY.reshape(10, 1)
trainY = trainY.reshape(140, 1)

In [5]:
#2. Build Model
    
class LinearRegression(nn.Module):
    def __init__(self):
        super().__init__() 
        self.weights = nn.Parameter(torch.randn(3, 1, dtype=torch.float, requires_grad=True))
        self.bias = nn.Parameter(torch.randn(1, dtype=torch.float, requires_grad=True))
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return torch.matmul(x, self.weights) + self.bias
    
# 1 - Almost everything in PyTorch is a nn.Module (think of this as neural network lego blocks)
# 2 - Start with random weights & biases, update with gradient descent
# 3 - "x" is the input data (e.g. training/testing features)

# - nn.Module: contains the larger building blocks (layers)
# - nn,Parameter: contains the smaller parameters like weights and biases (put these together to make nn.Module(s))
# - torch.optim: contains optimization methods on how to improve the parameters within nn.Parameter to better represent input data
# - def forward(): tells the larger blocks how to make calculations on inputs (tensors full of data) within nn.Module(s)

In [6]:
# Set manual seed since nn.Parameter are randomly initialzied
torch.manual_seed(42)

# Create an instance of the model (this is a subclass of nn.Module that contains nn.Parameter(s))
model_0 = LinearRegression()

# Check the nn.Parameter(s) within the nn.Module subclass we created
list(model_0.parameters())

[Parameter containing:
 tensor([[0.3367],
         [0.1288],
         [0.2345]], requires_grad=True),
 Parameter containing:
 tensor([0.2303], requires_grad=True)]

In [7]:
# List named parameters 
model_0.state_dict()

OrderedDict([('weights',
              tensor([[0.3367],
                      [0.1288],
                      [0.2345]])),
             ('bias', tensor([0.2303]))])

In [8]:
# 3. Training & Testing

# Training Loop
#    1. Forward Pass
#    2. Calculate Loss
#    3. Zero Gradients
#    4. Perform Backpropogation
#    5. Update Optimizer

# Testing Loop
#    1. Forward Pass
#    2. Calculate the Loss
#    3. Calculate Evaluation Metrics (Optional)

# Create the loss function
loss_fn = nn.L1Loss() # MAE loss is same as L1Loss

# Create the optimizer
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.01) 
# learning rate (how much the optimizer should change parameters at each step, higher=more (less stable), lower=less (might take a long time))


In [9]:
torch.manual_seed(42)

# Set the number of epochs (how many times the model will pass over the training data)
epochs = 100

# Create empty loss lists to track values
train_loss_values = []
test_loss_values = []
epoch_count = []

for epoch in range(epochs):
    ### Training

    model_0.train()                    # Put model in training mode (this is the default state of a model)

    y_pred = model_0(trainX)           # 1. Forward pass on train data using the forward() method inside 
    loss = loss_fn(y_pred, trainY)     # 2. Calculate the loss (how different are our models predictions to the ground truth)    
    optimizer.zero_grad()              # 3. Zero grad (Reset) of the optimizer
    loss.backward()                    # 4. Loss backwards
    optimizer.step()                   # 5. Progress the optimizer

    
    
    ### Testing
    
    model_0.eval()                     # Put the model in evaluation mode

    with torch.inference_mode():
      test_pred = model_0(testX)                                  # 1. Forward pass on test data
      test_loss = loss_fn(test_pred, testY.type(torch.float))     # 2. Caculate loss on test data
    
      # Print out what's happening
      if epoch % 10 == 0:
            epoch_count.append(epoch)
            train_loss_values.append(loss.detach().numpy())
            test_loss_values.append(test_loss.detach().numpy())
            print(f"Epoch: {epoch} | MAE Train Loss: {loss} | MAE Test Loss: {test_loss} ")

Epoch: 0 | MAE Train Loss: 2.2854599952697754 | MAE Test Loss: 1.5164395570755005 
Epoch: 10 | MAE Train Loss: 0.3803354501724243 | MAE Test Loss: 0.3829319477081299 
Epoch: 20 | MAE Train Loss: 0.28623056411743164 | MAE Test Loss: 0.3263406753540039 
Epoch: 30 | MAE Train Loss: 0.21582379937171936 | MAE Test Loss: 0.2866823077201843 
Epoch: 40 | MAE Train Loss: 0.20695090293884277 | MAE Test Loss: 0.2546616196632385 
Epoch: 50 | MAE Train Loss: 0.18511097133159637 | MAE Test Loss: 0.23452866077423096 
Epoch: 60 | MAE Train Loss: 0.17621785402297974 | MAE Test Loss: 0.2284967005252838 
Epoch: 70 | MAE Train Loss: 0.18369396030902863 | MAE Test Loss: 0.22306446731090546 
Epoch: 80 | MAE Train Loss: 0.1892288476228714 | MAE Test Loss: 0.2310035526752472 
Epoch: 90 | MAE Train Loss: 0.1819477677345276 | MAE Test Loss: 0.21811088919639587 


In [10]:
test_pred

tensor([[2.2708],
        [0.2662],
        [1.6153],
        [1.8089],
        [2.0821],
        [1.5300],
        [1.2739],
        [1.8654],
        [1.3508],
        [0.3125]])

In [11]:
testY

tensor([[1.8000],
        [0.2000],
        [1.7000],
        [2.0000],
        [2.3000],
        [1.5000],
        [1.0000],
        [2.3000],
        [1.2000],
        [0.2000]])

In [12]:
# Saving PyTorch Models
#    - torch.nn.Module.load_state_dict (Recommended)
#    - torch.save (Pickle)
#    - torch.load (Pickle)

from pathlib import Path

# 1. Create models directory 
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path 
MODEL_NAME = "01_pytorch_workflow_model_0.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict 
torch.save(obj=model_0.state_dict(),f=MODEL_SAVE_PATH)


In [13]:
#Loading PyTorch Models

loaded_model_0 = LinearRegression()
loaded_model_0.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

<All keys matched successfully>